In [12]:
from nltk import sem
import copy

In [119]:
class DefiniteClause:
    
    def __init__(self, head, body):
        self.head = head
        self.body = body
        
    def __repr__(self):
        return f"{self.head} <- {', '.join(map(str, self.body))}"
    
    def substitution(self, theta):

        new_head = self.head.substitution(theta)
        new_body = [atom.substitution(theta) for atom in self.body]

        return DefiniteClause(new_head, new_body)
    
    def is_ground(self):
        return self.head.is_ground() and all([atom.is_ground() for atom in self.body])

    def get_possible_substitutions(self, atoms):

        body = copy.deepcopy(self.body)
        outputs = _body_recursion(atoms, body, {})
        # print(outputs)
        return outputs
    
    def get_consequences(self, atoms):
        for substitution in self.get_possible_substitutions(atoms):
            # print(substitution)
            update = self.substitution(substitution).head
            # print(update)
            yield update
        
        
def _body_recursion(atoms, stack, substitution):
    # print('body_recursion w', atoms, stack, substitution)
    if len(stack) == 0:
        # print('returning', substitution)
        return [substitution]
    else:
        next_body = stack.pop()
        next_body = next_body.substitution(substitution)
        # print('next body:', next_body)
        candidates = [atom for atom in atoms if atom.pred == next_body.pred]
        # print('candidates', candidates)
        output = []
        for candidate in candidates:
            # print('processing', candidate)
            substitution_candidate = copy.deepcopy(substitution)
            # print('terms', list(zip(candidate.terms, next_body.terms)))
            # print([isinstance(b_term, Variable) or b_term == c_term for c_term, b_term in zip(candidate.terms, next_body.terms)])
            if all([isinstance(b_term, Variable) or b_term == c_term for c_term, b_term in zip(candidate.terms, next_body.terms)]):
                substitution_candidate.update({b:a for a, b in zip(candidate.terms, next_body.terms) if isinstance(b, Variable)})
                # print('substitution candidate', substitution_candidate)
                output += _body_recursion(atoms, copy.deepcopy(stack), substitution_candidate)
                
        # print('output', output)
        return output
    
class Variable:
    
    def __init__(self, name):
        self.name = name
    
    def __repr__(self):
        return self.name
    
    def __eq__(self, other):
        return self.name == other.name
    
    def __hash__(self):
        return hash(self.name)

class Constant:
    
    def __init__(self, name):
        self.name = name
    
    def __repr__(self):
        return self.name
    
    def __eq__(self, other):
        return self.name == other.name
    
    def __hash__(self):
        return hash(self.name)

class Predicate:
    
    def __init__(self, name, arity):
        self.name = name
        self.arity = arity
    
    def __repr__(self):
        return f"{self.name}/{self.arity}"
    
    def __eq__(self, other):
        return self.name == other.name and self.arity == other.arity
    
class Atom:
    
    def __init__(self, pred, terms):
        assert(len(terms) == pred.arity)
        self.pred = pred
        self.terms = terms
    
    def __repr__(self):
        return f"{self.pred.name}({', '.join(map(str, self.terms))})"
    
    def is_ground(self):
        return all([isinstance(t, Constant) for t in self.terms])
    
    def substitution(self, theta):
        new_terms = [theta[t] if t in theta else t for t in self.terms]
        return Atom(self.pred, new_terms)
     
    def __eq__(self, other):
        return self.pred == other.pred and self.terms == other.terms
    
    def __hash__(self):
        return hash(str(self.pred) + ','.join(map(str, self.terms)))
    
    @classmethod
    def from_string(cls, pred_name, pred_arity, terms):
        pred = Predicate(pred_name, pred_arity)
        terms = [Variable(t) if t.isupper() else Constant(t) for t in terms]
        return Atom(pred, terms)

def cn(rules, ground_atoms):
    out = list(ground_atoms)
    for rule in rules:
        if isinstance(rule, Atom) and rule.is_ground():
            out.append(rule)
        elif isinstance(rule, DefiniteClause):
            out += rule.get_consequences(ground_atoms)
    return set(out)

def consequences(rules):
    c0 = set()
    c1 = cn(rules, c0)
    while len(c1 - c0) > 0:
        c0 = c1
        c1 = cn(rules, c0)
    return c1

x = Constant('x')
y = Constant('y')
z = Constant('z')
a = Constant('a')
b = Constant('b')
c = Constant('c')

X = Variable('X')
Y = Variable('Y')
Z = Variable('Z')

on = Predicate('on', 2)
connected = Predicate('connected', 2)
edge = Predicate('edge', 2)

connectedXY = Atom(connected, [X, Y])
edgeXY = Atom(edge, [X, Y])
edgeXZ = Atom(edge, [X, Z])
connectedZY = Atom(connected, [Z, Y])

edgexy = Atom(edge, [x, y])
edgeab = Atom(edge, [a, b])
edgexb = Atom(edge, [x, b])
edgebc = Atom(edge, [b, c])
edgeca = Atom(edge, [c, a])
connectedbc = Atom(connected, [b, c])

connected_clause1 = DefiniteClause(connectedXY, [edgeXY])
connected_clause2 = DefiniteClause(connectedXY, [edgeXZ, connectedZY])

substitution = {X: a, Y: b, Z: c}

print(connected_clause2.substitution(substitution))

# print(connected_clause1)
# print('is ground?', connected_clause1.is_ground())
# subclause = connected_clause1.substitution(substitution)
# print(subclause)
# print('is ground?', subclause.is_ground())

# print(connected_clause1.get_possible_substitutions([edgexy]))
# print(connected_clause1.get_possible_substitutions([edgeab, edgexb]))

# print(list(connected_clause1.get_consequences([edgeab, edgexb])))
# print(connected_clause2)
# print('clause 2 consequences', list(connected_clause2.get_consequences([edgeab, connectedbc])))

R = {edgeab, edgebc, edgeca, connected_clause1, connected_clause2}

print(R)

a1 = cn(R, {})
print('###########cr1', a1)

a2 = cn(R, a1)
print('###########cr2', a2 - a1)



a3 = cn(R, a2)
print('###########cr3', a3 - a2)

a4 = cn(R, a3)
print('###########cr4', a4 - a3)

a5 = cn(R, a4)
print('###########cr5', a5 - a4)

consequences(R)

connected(a, b) <- edge(a, c), connected(c, b)
{connected(X, Y) <- edge(X, Y), edge(a, b), edge(b, c), connected(X, Y) <- edge(X, Z), connected(Z, Y), edge(c, a)}
###########cr1 {edge(c, a), edge(b, c), edge(a, b)}
###########cr2 {connected(a, b), connected(c, a), connected(b, c)}
###########cr3 {connected(b, a), connected(a, c), connected(c, b)}
###########cr4 {connected(b, b), connected(a, a), connected(c, c)}
###########cr5 set()


{connected(a, a),
 connected(a, b),
 connected(a, c),
 connected(b, a),
 connected(b, b),
 connected(b, c),
 connected(c, a),
 connected(c, b),
 connected(c, c),
 edge(a, b),
 edge(b, c),
 edge(c, a)}

In [124]:
zero0 = Atom.from_string('zero', 1, ['0'])
B = [zero0]
for i, j in zip(range(0, 100), range(1, 101)):
    B.append(Atom.from_string('succ', 2, [str(i), str(j)]))


P = []
for i in range(0, 101, 2):
    P.append(Atom.from_string('even', 1, [str(i)]))
    
N = []
for i in range(1, 101, 2):
    N.append(Atom.from_string('even', 1, [str(i)]))

In [125]:
r1 = DefiniteClause(Atom.from_string('even', 1, ['X']), [Atom.from_string('zero', 1, ['X'])])
r2 = DefiniteClause(Atom.from_string('even', 1, ['X']), [Atom.from_string('even', 1, ['Y']), Atom.from_string('succ2', 2, ['Y', 'X'])])
r3 = DefiniteClause(Atom.from_string('succ2', 2, ['X', 'Y']), [Atom.from_string('succ', 2, ['X', 'Z']), Atom.from_string('succ', 2, ['Z', 'Y'])])

In [126]:
R = B + [r1, r2, r3]
consequences(R) 

{even(0),
 even(10),
 even(100),
 even(12),
 even(14),
 even(16),
 even(18),
 even(2),
 even(20),
 even(22),
 even(24),
 even(26),
 even(28),
 even(30),
 even(32),
 even(34),
 even(36),
 even(38),
 even(4),
 even(40),
 even(42),
 even(44),
 even(46),
 even(48),
 even(50),
 even(52),
 even(54),
 even(56),
 even(58),
 even(6),
 even(60),
 even(62),
 even(64),
 even(66),
 even(68),
 even(70),
 even(72),
 even(74),
 even(76),
 even(78),
 even(8),
 even(80),
 even(82),
 even(84),
 even(86),
 even(88),
 even(90),
 even(92),
 even(94),
 even(96),
 even(98),
 succ(0, 1),
 succ(1, 2),
 succ(10, 11),
 succ(11, 12),
 succ(12, 13),
 succ(13, 14),
 succ(14, 15),
 succ(15, 16),
 succ(16, 17),
 succ(17, 18),
 succ(18, 19),
 succ(19, 20),
 succ(2, 3),
 succ(20, 21),
 succ(21, 22),
 succ(22, 23),
 succ(23, 24),
 succ(24, 25),
 succ(25, 26),
 succ(26, 27),
 succ(27, 28),
 succ(28, 29),
 succ(29, 30),
 succ(3, 4),
 succ(30, 31),
 succ(31, 32),
 succ(32, 33),
 succ(33, 34),
 succ(34, 35),
 succ(35, 36),
 

In [117]:
type(B[0].terms[0])

__main__.Variable

In [118]:
'0'.isupper()

False